In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
rng = np.random.RandomState(42)

In [2]:
index = [('California', 2000), ('California', 2010),
                    ('New York', 2000), ('New York', 2010),
                    ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
                          18976457, 19378102,
                          20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [4]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           codes=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

In [5]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [8]:
pd.DataFrame(pop,columns = ["population"])

population
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

In [9]:
pop.unstack()

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [11]:
pop_df = pd.DataFrame({"total":pop,
                      "under18":[9267089,9284094,
                                4687374, 4318033,
                                 5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [12]:
pop_df.unstack()

total            under18         
                2000      2010     2000     2010
California  33871648  37253956  9267089  9284094
New York    18976457  19378102  4687374  4318033
Texas       20851820  25145561  5906301  6879014

In [15]:
pd.MultiIndex.from_arrays([list("aabb"),[1,2,1,2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [22]:
index = pd.MultiIndex.from_product([["a","b"],[1,2]])
pd.DataFrame({"popu":1},index = index)

popu
a 1     1
  2     1
b 1     1
  2     1

MultiIndex level names

In [24]:
pop.index.names = ['state','year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [25]:
index = pd.MultiIndex.from_product([[2013,2014],[1,2]],
                                  names = ['year','visit'])
columns = pd.MultiIndex.from_product([['Bob','Guido','Sue'],['HR','Temp']],
                                    names = ['subject','type'])
data = np.round(np.random.randn(4,6),1)
data[:,::2] *= 10
data += 37

health_data = pd.DataFrame(data,index = index, columns = columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      46.0  36.7  56.0  37.4  32.0  36.8
     2      27.0  36.5  35.0  35.9  41.0  36.6
2014 1      33.0  37.4  50.0  37.9  24.0  35.8
     2      26.0  37.6  43.0  37.4  28.0  38.3

In [26]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      56.0  37.4
     2      35.0  35.9
2014 1      50.0  37.9
     2      43.0  37.4

## Indexing and Slicing a MultiIndex

In [27]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [30]:
pop["California"]

year
2000    33871648
2010    37253956
dtype: int64

In [29]:
pop[['California']]

state       year
California  2000    33871648
            2010    37253956
dtype: int64

#### Multiply indexed DataFrames

In [31]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      46.0  36.7  56.0  37.4  32.0  36.8
     2      27.0  36.5  35.0  35.9  41.0  36.6
2014 1      33.0  37.4  50.0  37.9  24.0  35.8
     2      26.0  37.6  43.0  37.4  28.0  38.3

In [35]:
health_data['Guido',"HR"]

year  visit
2013  1        56.0
      2        35.0
2014  1        50.0
      2        43.0
Name: (Guido, HR), dtype: float64

In [38]:
health_data.iloc[:2,:2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      46.0  36.7
     2      27.0  36.5

In [39]:
health_data.loc[:,('Bob','HR')]

year  visit
2013  1        46.0
      2        27.0
2014  1        33.0
      2        26.0
Name: (Bob, HR), dtype: float64

In [40]:
health_data.loc[(:,1),(:,"HR")]

SyntaxError: invalid syntax (<ipython-input-40-f2bb13ac1323>, line 1)

In [43]:
health_data.loc[pd.IndexSlice[:,1], pd.IndexSlice[:,'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,46.0,56.0,32.0
2014,1,33.0,50.0,24.0


In [60]:
health_data.reset_index()[health_data.reset_index().visit == 1].set_index(["year","visit"])["Guido"]

,type,HR,Temp
year,visit,,
2013,1,56.0,37.4
2014,1,50.0,37.9


#### Rearranging Multi-Indices

In [62]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.836225
      2      0.105229
c     1      0.301919
      2      0.853115
b     1      0.654773
      2      0.178013
dtype: float64

In [64]:
data = data.sort_index()
data

char  int
a     1      0.836225
      2      0.105229
b     1      0.654773
      2      0.178013
c     1      0.301919
      2      0.853115
dtype: float64

In [67]:
 pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [70]:
pop.unstack()

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [68]:
pop.unstack(level = 0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [69]:
pop.unstack(level = 1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


#### Index setting and resetting

In [74]:
pop_flat = pop.reset_index(name = "population")
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [75]:
pop_flat.set_index(['state','year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

# Data Aggregations on Multi-Indices

In [76]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      46.0  36.7  56.0  37.4  32.0  36.8
     2      27.0  36.5  35.0  35.9  41.0  36.6
2014 1      33.0  37.4  50.0  37.9  24.0  35.8
     2      26.0  37.6  43.0  37.4  28.0  38.3

In [77]:
data_mean = health_data.mean(level='year')
data_mean

subject   Bob       Guido          Sue       
type       HR  Temp    HR   Temp    HR   Temp
year                                         
2013     36.5  36.6  45.5  36.65  36.5  36.70
2014     29.5  37.5  46.5  37.65  26.0  37.05

In [79]:
health_data.groupby("year").mean()

subject   Bob       Guido          Sue       
type       HR  Temp    HR   Temp    HR   Temp
year                                         
2013     36.5  36.6  45.5  36.65  36.5  36.70
2014     29.5  37.5  46.5  37.65  26.0  37.05

In [88]:
data_mean.mean(axis = 1,level = "subject")

subject,Bob,Guido,Sue
year,,,
2013,36.55,41.075,36.600
2014,33.50,42.075,31.525


In [87]:
data_mean.mean(axis = 1)

year
2013    38.075
2014    35.700
dtype: float64